In [108]:
import configparser

import pandas as pd
import psycopg2

config = configparser.ConfigParser()
config.read("dwh.cfg")
print(*config["REDSHIFT"].values())

'dwhcluster.cu0kxfsouwy2.us-west-2.redshift.amazonaws.com' dwh dwhuser Passw0rd 5439


In [109]:
connection = psycopg2.connect(
    "host={} dbname={} user={} password={} port={}".format(
        *config["REDSHIFT"].values()
    )
)
cursor = connection.cursor()
query = """
WITH 
    user_log_data AS (
        SELECT
            userId AS user_id,
            firstName AS first_name,
            lastName AS last_name,
            gender,
            level,
            TIMESTAMP 'epoch' + ts/1000 * INTERVAL '1 second' AS time
        FROM
            log_data
        WHERE
            auth = 'Logged In' AND
            length > 0
    ), 
    max_user_times AS (
        SELECT
            user_id,
            MAX(time) AS max_time
        FROM
            user_log_data
        GROUP BY
            user_id
    )
SELECT
    user_log_data.user_id,
    user_log_data.first_name,
    user_log_data.last_name,
    user_log_data.gender,
    user_log_data.level
FROM
    user_log_data
JOIN
    max_user_times
ON
    user_log_data.user_id = max_user_times.user_id AND
    user_log_data.time = max_user_times.max_time;
"""

cursor.execute(query)

rows = cursor.fetchall()
for row in rows:
    print(row)

(20, 'Aiden', 'Ramirez', 'M', 'paid')
(84, 'Shakira', 'Hunt', 'F', 'free')
(3, 'Isaac', 'Valdez', 'M', 'free')
(59, 'Lily', 'Cooper', 'F', 'free')
(7, 'Adelyn', 'Jordan', 'F', 'free')
(2, 'Jizelle', 'Benjamin', 'F', 'free')
(45, 'Dominick', 'Norris', 'M', 'free')
(70, 'Jaleah', 'Hayes', 'F', 'paid')
(71, 'Ayleen', 'Wise', 'F', 'free')
(27, 'Carlos', 'Carter', 'M', 'free')
(51, 'Maia', 'Burke', 'F', 'free')
(25, 'Jayden', 'Graves', 'M', 'paid')
(42, 'Harper', 'Barrett', 'M', 'paid')
(41, 'Brayden', 'Clark', 'M', 'free')
(35, 'Molly', 'Taylor', 'F', 'free')
(81, 'Sienna', 'Colon', 'F', 'free')
(18, 'Jacob', 'Rogers', 'M', 'free')
(64, 'Hannah', 'Calhoun', 'F', 'free')
(99, 'Ann', 'Banks', 'F', 'free')
(65, 'Amiya', 'Davidson', 'F', 'paid')
(34, 'Evelin', 'Ayala', 'F', 'free')
(62, 'Connar', 'Moreno', 'M', 'free')
(38, 'Gianna', 'Jones', 'F', 'free')
(8, 'Kaylee', 'Summers', 'F', 'free')
(66, 'Kevin', 'Arellano', 'M', 'free')
(48, 'Marina', 'Sutton', 'F', 'free')
(56, 'Cienna', 'Freeman',

In [110]:
insert_query = """
INSERT INTO users (user_id, first_name, last_name, gender, level)
VALUES (%s, %s, %s, %s, %s)
"""

for row in rows:
    cursor.execute(insert_query, row)

In [111]:
connection = psycopg2.connect(
    "host={} dbname={} user={} password={} port={}".format(
        *config["REDSHIFT"].values()
    )
)
cursor = connection.cursor()
query = """
WITH 
    user_log_data AS (
        SELECT
            userId AS user_id,
            firstName AS first_name,
            lastName AS last_name,
            gender,
            level,
            TIMESTAMP 'epoch' + ts/1000 * INTERVAL '1 second' AS time
        FROM
            log_data
        WHERE
            auth = 'Logged In' AND
            length > 0
    ), 
    max_user_times AS (
        SELECT
            user_id,
            MAX(time) AS max_time
        FROM
            user_log_data
        GROUP BY
            user_id
    )
-- Add a space here
INSERT INTO users
SELECT
    user_log_data.user_id,
    user_log_data.first_name,
    user_log_data.last_name,
    user_log_data.gender,
    user_log_data.level
FROM
    user_log_data
JOIN
    max_user_times
ON
    user_log_data.user_id = max_user_times.user_id AND
    user_log_data.time = max_user_times.max_time;
"""

cursor.execute(query)

rows = cursor.fetchall()
for row in rows:
    print(row)

SyntaxError: syntax error at or near "INSERT"
LINE 27: INSERT INTO users
         ^
